In [1]:
# Crear el notebook completo en un archivo .ipynb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer

# Cargar datos
costos_df = pd.read_excel('Costos_Subprocesos.xlsx')
indicadores_df = pd.read_excel('Indicadores_Perforación.xlsx')

# Convertir las fechas a formato datetime
costos_df['Subproceso_Costo'] = pd.to_datetime(costos_df['Subproceso_Costo'])
indicadores_df['Mina/Rajo_Flota_Indicador'] = pd.to_datetime(indicadores_df['Mina/Rajo_Flota_Indicador'])

# Unir los DataFrames por la columna de fechas
df_unificado = pd.merge(costos_df, indicadores_df, left_on='Subproceso_Costo', right_on='Mina/Rajo_Flota_Indicador', how='inner')
df_unificado = df_unificado.drop(columns=['Mina/Rajo_Flota_Indicador'])

# Calcular la correlación entre todos los subprocesos y los indicadores
subprocesos_cols = costos_df.columns[1:]  # Excluyendo la columna de fecha
indicadores_cols = indicadores_df.columns[1:]  # Excluyendo la columna de fecha
correlation_matrix = df_unificado.corr()
correlation_filtered = correlation_matrix.loc[subprocesos_cols, indicadores_cols]
top_correlations = correlation_filtered.apply(lambda x: x.nlargest(3), axis=1)

# Dividir los datos en entrenamiento (2016-2021) y prueba (2022)
train_data = df_unificado[df_unificado['Subproceso_Costo'] < '2022-01-01']
test_data = df_unificado[df_unificado['Subproceso_Costo'] == '2022-01-01']

In [2]:
# Modelo 1: Linear Regression (Múltiple)
resultados_lr = {}
lr_model = LinearRegression()
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        
        # Eliminar filas con NaN en X_train y y_train
        X_y_train = pd.concat([X_train, y_train], axis=1).dropna()
        X_train = X_y_train.iloc[:, :-1].values
        y_train = X_y_train.iloc[:, -1].values
        
        # Eliminar filas con NaN en X_test
        X_test = X_test.dropna().values

        # Entrenar el modelo
        lr_model.fit(X_train, y_train)
        
        # Realizar la predicción para el 2022
        y_pred_2022 = lr_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_lr[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}

resultados_lr_df = pd.DataFrame(resultados_lr).T
print("Resultados para Linear Regression (Múltiple):")
print(resultados_lr_df)

Resultados para Linear Regression (Múltiple):
                                        Predicción 2022  Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP              1695594.857562              0.0   
ADM OPERACIÓN Y GESTIÓN ESP SUR             8927.441737         30.27672   
ADM OPERACIÓN Y GESTIÓN MIR               160701.379119              0.0   
ADM OPERACIÓN Y GESTIÓN OXE              -464310.785967              0.0   
ADM OPERACIÓN Y GESTIÓN TC                170370.255256        906.14448   
ADM OPERACIÓN Y GESTIÓN TES SUR           116776.595251              0.0   
ADMINISTRACIÓN DESARROLLO MINA              1155.486889     881286.83064   
ADMINISTRACIÓN MINA                        -8620.083726    1780999.25816   
CARGUÍO                                 -1267324.063292     833630.60568   
CARGUÍO ENCUENTRO                           -15193.3529         262.9632   
CARGUÍO ESPERANZA                        2416230.982762              0.0   
CARGUÍO ESPERANZA SUR                     

In [3]:
# Modelo 2: Polynomial Regression (Regresión Polinómica)
resultados_pr = {}
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        
        poly = PolynomialFeatures(degree=2)
        X_train_poly = poly.fit_transform(X_train)
        X_test_poly = poly.transform(X_test)
        
        pr_model = LinearRegression()
        pr_model.fit(X_train_poly, y_train)
        y_pred_2022 = pr_model.predict(X_test_poly)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_pr[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_pr_df = pd.DataFrame(resultados_pr).T
print("Resultados para Polynomial Regression (Regresión Polinómica):")
print(resultados_pr_df)

ValueError: Input X contains NaN.
PolynomialFeatures does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Modelo 3: Neural Network (Redes Neuronales)
resultados_nn = {}
nn_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        nn_model.fit(X_train, y_train)
        y_pred_2022 = nn_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_nn[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_nn_df = pd.DataFrame(resultados_nn).T
print("Resultados para Neural Network (Redes Neuronales):")
print(resultados_nn_df)

In [ ]:
# Modelo 4: Random Forest
resultados_rf = {}
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        rf_model.fit(X_train, y_train)
        y_pred_2022 = rf_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_rf[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_rf_df = pd.DataFrame(resultados_rf).T
print("Resultados para Random Forest:")
print(resultados_rf_df)

In [ ]:
# Modelo 5: Gradient Boosting
resultados_gb = {}
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        gb_model.fit(X_train, y_train)
        y_pred_2022 = gb_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_gb[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_gb_df = pd.DataFrame(resultados_gb).T
print("Resultados para Gradient Boosting:")
print(resultados_gb_df)

In [ ]:
# Modelo 6: XGBoost
resultados_xgb = {}
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        xgb_model.fit(X_train, y_train)
        y_pred_2022 = xgb_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_xgb[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_xgb_df = pd.DataFrame(resultados_xgb).T
print("Resultados para XGBoost:")
print(resultados_xgb_df)

In [ ]:
# Comparación de resultados
resultados_totales = {
    'Linear Regression (Múltiple)': resultados_lr_df,
    'Polynomial Regression': resultados_pr_df,
    'Neural Network': resultados_nn_df,
    'Random Forest': resultados_rf_df,
    'Gradient Boosting': resultados_gb_df,
    'XGBoost': resultados_xgb_df
}
for modelo, resultados in resultados_totales.items():
    print(f"Resumen para {modelo}:")
    print(resultados[['Diferencia %']])
    print("\n\n")

In [ ]:
import pandas as pd
import numpy as np

# Resultados de los modelos (simula los DataFrames que imprimiste anteriormente)
resultados = {
    'Linear Regression': resultados_lr_df,
    'Polynomial Regression': resultados_pr_df,
    'Neural Network': resultados_nn_df,
    'Random Forest': resultados_rf_df,
    'Gradient Boosting': resultados_gb_df,
    'XGBoost': resultados_xgb_df
}

# Crear un DataFrame para almacenar los mejores modelos para cada subproceso
mejor_modelo_df = pd.DataFrame(columns=['Subproceso', 'Mejor Modelo', 'Diferencia %'])

# Comparar los resultados para cada subproceso
for subproceso in resultados['Linear Regression'].index:
    mejor_modelo = None
    menor_diferencia = np.inf
    
    for modelo, df in resultados.items():
        if subproceso in df.index and df.loc[subproceso, 'Diferencia %'] != 'N/A':
            diferencia = abs(df.loc[subproceso, 'Diferencia %'])
            if diferencia < menor_diferencia:
                menor_diferencia = diferencia
                mejor_modelo = modelo
    
    # Guardar el mejor modelo para el subproceso
    nueva_fila = pd.DataFrame({
        'Subproceso': [subproceso],
        'Mejor Modelo': [mejor_modelo],
        'Diferencia %': [menor_diferencia]
    })
    
    mejor_modelo_df = pd.concat([mejor_modelo_df, nueva_fila], ignore_index=True)

# Mostrar los resultados
print(mejor_modelo_df)
